In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 選擇權之避險參數

In [8]:
def BSOpValuation(opt, S, K, r, q, T, sigma):
    import numpy as np
    from scipy import stats
    
    act = np.exp(-r * T)
    actd = np.exp(-q * T)
    S=float(S)
    
    if S > 0 and K > 0 and T > 0 and sigma > 0:
        Done = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        Dtwo = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        
        Ndone = stats.norm.cdf(opt * Done, 0.0, 1.0)
        Ndtwo = stats.norm.cdf(opt * Dtwo, 0.0, 1.0)
        
        return opt * (S * actd * Ndone - K * act * Ndtwo)

    elif S > 0 and K > 0 and sigma > 0 and T == 0:
        return np.maximum(0, opt * (S - K))
    
    else:
        print("Warning : Share price(S) and Exercice price(X) and time to maturity(tyr) and volatility(sigma) must be positive")
        
def BSDOne(S, K, r, q, T, sigma):
    import numpy as np
    return (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

def BSDTwo(S, K, r, q, T, sigma):
    import numpy as np
    return (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

def BSNdashOne(S, K, r, q, T, sigma):
    import numpy as np
    return (np.exp(-(BSDOne(S, K, r, q, T, sigma)) ** 2 / 2) / np.sqrt(2 * np.pi))

In [6]:
def BSValGreeks(greek, opt, S, K, r, q, T, sigma):
    from scipy import stats
    import numpy as np
    act = np.exp(-q * T)
    
    c = BSOpValuation(opt, S, K, r, q, T, sigma)
    c1 = stats.norm.cdf(opt * BSDOne(S, K, r, q, T, sigma))
    c1d = BSNdashOne(S, K, r, q, T, sigma)
    c2 = stats.norm.cdf(opt * BSDTwo(S, K, r, q, T, sigma))
    d = opt * act * c1
    g = c1d * act / (S * sigma * np.sqrt(T))
    
    if greek == 1: #1 = delta
        return d
    elif greek == 2: #2 = gamma
        return g
    elif greek == 3: #3 = rho
        return opt * K * T * np.exp(-r * T) * c2
    elif greek == 4: #4 = theta
        return r * c - (r - q) * S * d - 0.5 * (sigma * S) ** 2 * g
    elif greek == 5: #5 = vega
        return S * np.sqrt(T) * c1d * act

In [9]:
#for example
greek1 = BSValGreeks(1,1,49,50,0.05,0,0.3846,20/100)
greek1

0.5216016339715761